In [1]:
# Import GeoPandas and Matplotlib libraries 
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gp

In [7]:
# Read the mining site data
mining_sites = geopandas.read_file('Mines\\ipis_cod_mines.geojson')
# Print the first rows and the CRS information
print(mining_sites.head())
print(mining_sites.crs)

   visit_date                                name  n_workers mineral  \
0  2013-03-27                           Mayi-Tatu      150.0    Gold   
1  2013-03-27                             Mabanga      115.0    Gold   
2  2013-03-27                             Molende      130.0    Gold   
3  2013-03-27                          Embouchure      135.0    Gold   
4  2013-03-27  Apumu-Atandele-Jerusalem-Luka Yayo      270.0    Gold   

                   geometry  
0  POINT (29.66033 1.01089)  
1  POINT (29.65862 1.00308)  
2  POINT (29.65629 0.98563)  
3  POINT (29.64494 0.99976)  
4  POINT (29.66000 0.95600)  
epsg:4326
